# Assignment Two from HW 3
## Author: Sofiya Elyukin
### Code taken from the Ctibikes_gender python notebook in the PUI2016 repository. The null hypothesis was  developed through a group discussion with Ben Alpert, Sebastian Bana, Jonathan Geis, Fernando Melchor, and Scott Smith.

### Research Idea:
    People using CitiBike during "rush hours" (specific time to be defined based on MTAinfo: 
    http://web.mta.info/nyct/subway/howto_sub.htm) are commuters, and therefore there would more commuters using  CitiBike on
    weekdays than weekends.

### Null Hypothesis:
    The quantity of CitiBike rides during weekday rush hours as a ratio of total rides for that day will be equal to or
    significantly less than the same ratio on Saturday or Sunday. I will use a significance level of α = 0.05, which means I
    want the probability of getting a result at least as significant as mine to be less then 5%.

### Alternative Hypothesis:
    The amount of CitiBike rides during weekday rush hours as a ratio of total rides for that day will be greater than the
    same ratio on Saturday or Sunday.

In [1]:
from __future__  import print_function, division
import pylab as pl
import pandas as pd
import numpy as np
import os
%pylab inline

if os.getenv ('PUI2016') is None:
    print ("Must set env variable PUI2016")
if os.getenv ('PUIDATA') is None:
    print ("Must set env variable PUI2016")
print (os.getenv('PUI2016'))
print (os.getenv('PUIDATA'))

Populating the interactive namespace from numpy and matplotlib
/home/cusp/se1296/PUI2016
/home/cusp/se1296/PUIdata


In [3]:
# downloaded data to home/cusp/se1296/PUI2016_se1296
df = pd.read_csv(os.getenv("PUI2016") +"_se1296" + "/201604-citibike-tripdata.csv")
#df.head(5)
print ('done')

done


In [4]:
df['date'] = pd.to_datetime(df['starttime'])
print ("done")
df.head(3)

done


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date
0,496,4/1/2016 00:00:26,4/1/2016 00:08:42,254,W 11 St & 6 Ave,40.735324,-73.998004,236,St Marks Pl & 2 Ave,40.728419,-73.987140,19704,Subscriber,1984.0,1,2016-04-01 00:00:26
1,104,4/1/2016 00:00:27,4/1/2016 00:02:12,311,Norfolk St & Broome St,40.717227,-73.988021,350,Clinton St & Grand St,40.715595,-73.987030,21555,Subscriber,1970.0,1,2016-04-01 00:00:27
2,128,4/1/2016 00:00:30,4/1/2016 00:02:38,3064,Myrtle Ave & Lewis Ave,40.696820,-73.937569,3059,Pulaski St & Marcus Garvey Blvd,40.693398,-73.939877,17430,Subscriber,1986.0,1,2016-04-01 00:00:30


In [ ]:
#!curl -O 'https://s3.amazonaws.com/tripdata/201604-citibike-tripdata.zip'
#os.system("unzip " + os.getenv("PUIDATA") + "/" + "201604" + "-citibike-tripdata.zip")
#df = pd.read_csv(os.getenv("PUIDATA") + "/201604-citibike-tripdata.csv") #file doesn't exist

In [ ]:

def getCitiBikeCSV(datestring):
    print ("Downloading", datestring)
    ### First I will heck that it is not already there
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        if os.path.isfile(datestring + "-citibike-tripdata.csv"):
            # if in the current dir just move it
            if os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA")):
                print ("Error moving file!, Please check!")
        #otherwise start looking for the zip file
        else:
            if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip"):
                if not os.path.isfile(datestring + "-citibike-tripdata.zip"):
                    os.system("curl -O https://s3.amazonaws.com/tripdata/" + datestring + "-citibike-tripdata.zip")
                ###  To move it I use the os.system() functions to run bash commands with arguments
                os.system("mv " + datestring + "-citibike-tripdata.zip " + os.getenv("PUIDATA"))
            ### unzip the csv 
            os.system("unzip " + os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip")
            ## NOTE: old csv citibike data had a different name structure. 
            if '2014' in datestring:
                os.system("mv " + datestring[:4] + '-' +  datestring[4:] + 
                          "\ -\ Citi\ Bike\ trip\ data.csv " + datestring + "-citibike-tripdata.csv")
            os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA"))
    ### One final check:
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        print ("WARNING!!! something is wrong: the file is not there!")

    else:
        print ("file in place, you can continue")

In [ ]:
datestring = '201604'
getCitiBikeCSV(datestring)